# 2. Emergency room service with prioritised queues 

## Introduction

### Learning objective:
- Estimation of M/M/c Poisson queues based on observed data 
- Use of G/G/c queues  
- Introduction of queue priorities for different user groups and the implied 
  consequence 
- Study different KPIs for queuing systems 
- Design an optimal service schedule based on requirements for service 
  guarantees 

Consider an emergency room at a hospital. People arriving to the emergency 
room will form a queue, which will be served based on a first-come first-serve
(FCFS) principle. There is up to n different counters with one shared queue.
Each counter is served by a team. 

The hospital has collected data during a period of time about arrivals and the
service of people in the queuing system. See Table 1 below.

| Variables | Description | Unit |
| --------- | ----------- | ---- |
| ID | ID of the person | ID |
| Age | Age of the person | Year |
| Critical | If the person needs critical treatment | 1 or 0 |
| Arrival | Minutes after midnight | Minutes, between 1-1440 |
| InterArrival | Time since arrival of last patient | Minutes |
| Prio | Priority of patient, higher number = higher priority (Q 6)  | 1, 2, 3 |
| TimePeriod | Which 8-hour time period patient is in (Q 7.a)  | 1, 2, 3|
| Category | Category variable for KPIs – 1 (prio group 1+2), 2 (prio 3) (Q 7.a)  | 1, 2 |
| Service | Time of the service in minutes  | Minutes |

### Python / jupyther kernel initialization

In [ ]:
!pip install -U simpy

In [ ]:
# It good practice to always import packages at the header of the your Python
# script and make sort them alphabetically, or using some clear classification
#
# Here, will load some useful packages, starting with the core Python packages
from pathlib import Path  # easy and cross platform resilient path definition

# And then proceeding to extension packages
import pandas as pd  # de-facto standard for dataframes
import numpy as np  # robust and fast numerics
import scipy.stats as stats  # applied numerics, including statistics
import scipy.special as special  # special mathematical functions
import simpy  # discrete event simulation library
import sklearn.metrics as metrics  # library of standard score functions, etc

# load plotting packages
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#
import math

# And finally setting important 'global' variables here too
path_0 = Path.cwd()

rng = np.random.default_rng(42)

cmap = cm.get_cmap('viridis')

# --------------------------------------------------------------------------- #
# Last, but not least, it is advisable to wrap code and comments to a maximum
# of 79 characters for a wide variaty of reasons, for details see:
# https://peps.python.org/pep-0008/#maximum-line-length

## A. Data inspection
Before proceeding with the assignment problems it is adviasable to inspect 
the data.

This is a guided inspection of the data, all code in this section is complete
and works as expected, but we strongly recommend students to play with it.

Despite the code being complete, students are expected to analyse the results
and write down their comments. We introduced a few cells for this purpose, but
we encourage you to comment as you see fit, adding more cells if necessary.

For each of reference add new cells in `Markdown` style with the following format:

### Exercise 0.: **Guided data inspection**

**_Student comments:_**

A sample comment.

#### **Loading data with Pandas**

In [ ]:
# Loading data
from google.colab import files
uploaded = files.upload()

In [ ]:
# importing data with pandas
# this dataset is saved in standard CSV format, so we load it into
# variable `df` as follows
from pandas.io.common import dataclasses

df = pd.read_csv(path_0 / 'AgentsFrame.csv')

# the number of rows and columns is read from the `shape` attribute
num_rows, num_cols = df.shape

# To print messages with variables in the middle of the message
# string, use f-strings as follows
print(f'The dataframe contains {num_rows} records with '
      f'{num_cols} entries per record.')

# a summary of the dataframe can be see with the `head` method
df.head()

#### **Inspection of Arrival times, intervals, service, IDs and Age**

Here you have to sort your data, find extreme values and search for possible
missing entries. 

To verify missing data, we force casting data to float. Missing, none and 
nans are reported as np.nan and can be accounted for this way.

In [ ]:
# --- CODE START --- #

# Find extreme value (min, max) and other summary data (mean, std, etc.)
print(df.describe())

In [ ]:
df = df.astype(float)
df.isnull().values.any()
# --- CODE END ----- #

**_Student comments:_**

*There are 424 entries for each variable (424 patients) \\
IDs do indeed range from 1 to 424 as well \\
The arrival times range from 3.764692 (min) to 1444.698142 (max) \\
The intervals have extreme values min 0.005660 and max 24.438476 (M = 3.407307, SD = 3.808549) \\
Service times range from 0.003503 to 185.524811 (M = 16.141066, SD = 20.354480) \\
The minimum age is 0 and the max is 94 (M = 55.540094, SD = 22.375229) \\
 \\
There are no missing values*

#### **Let us now inspect the data with restricted number of options**

Verify the values that you can find in each column and how frequent
they appear.

In [ ]:
# --- CODE START --- #
pd.set_option('display.max_rows', None)

#df['AgentArrival'].value_counts().sort_index()
#df['ID'].value_counts().sort_index()
#df['Critical'].value_counts().sort_index()
#df['Age'].value_counts().sort_index()
#df['InterArrival'].value_counts().sort_index()
#df['Service'].value_counts().sort_index()
#df['Prio'].value_counts().sort_index()
#df['TimePeriod'].value_counts().sort_index()
#df['Category'].value_counts().sort_index()



In [ ]:
#Graph for age:
ax = df['Age'].value_counts().sort_index().plot(kind='bar', 
                                    figsize=(20,8),
                                    title="Frequency of age values in dataset")
ax.set_xlabel("Age")
ax.set_ylabel("Frequency")

# --- CODE END ----- #

**_Student comments:_**

*AgentArrival, ID, InterArrival, Service have all (424) unique values \\
Critical has two options: 0 or 1. The value 0 is present 352 times and 1 is present 72 times. \\
Age has many different values, but some are there more often than others (Let's check this again when making the report?) \\
Prio has options 1, 2 or 3. We have 325 observations of 1, 27 observations of 2 and 72 observations of value 3. \\
Timeperiod can be a 1 (89 times), 2 (165 times), 3 (169 times), or a 4 (1 time). \\
Category can have value 1 (352 times) or 2 (72 times)*

You should have found a time period outside the range 1-3. 
Can you identify and determine the correct period for that? Hint: remainder

In [ ]:
# --- CODE START --- #
print(df.loc[df['TimePeriod'] == 4])


**_Student comments:_**

*The ID of the person with the "wrong" timeperid is 424 and their arrival time is 1444.698142. The max arrival time should be 1440, so we substract that from the number here, which leaves us 4.698142 minutes. The correct TimePeriod should, thus, be 1, because it is in the first eight hours after midnight.*

Given the agent arrival time, we can clearly see that this is simply a case
of agent with arrival time at the begining of the next day, hence time period
4 is the time period 1 for the next day. We can create modulus 3 TimePeriod
that ensures uniform representation.

In [ ]:
df['TimePeriod'] = df['TimePeriod'] % 3
# --- CODE END ----- #

In [ ]:
# we make us of DataFrame method `isin` to get an array
# of bool that identifies where we find what we are searching
#
# By defining these boolean arrays, we can then easily extract
# data that matches the desired characteristics
#
idx_crit = df['Critical'].isin([1]).values
idx_cat = df['Category'].isin([2]).values

idx_prio_1 = df['Prio'].isin([1]).values
idx_prio_2 = df['Prio'].isin([2]).values
idx_prio_3 = df['Prio'].isin([3]).values

# and we can perform boolean operations as well
idx_prio_12 = np.logical_or(idx_prio_1, idx_prio_2)

# Applying numpy sum to an array of bool simply returns the total
# number of True values
t_crit_cat = np.sum(idx_crit == idx_cat)
t_crit_prio_3 = np.sum(idx_crit == idx_prio_3)

# the ~ is the logic negation operator in Python
t_crit_prio_2 = np.sum(~idx_crit[idx_prio_2])
t_crit_prio_1 = np.sum(~idx_crit[idx_prio_1])
t_crit_prio_12 = np.sum(~idx_crit[idx_prio_12])

num_prio_2 = np.sum(idx_prio_2)
num_prio_1 = np.sum(idx_prio_1)
num_prio_12 = np.sum(idx_prio_12)

tests = [
    t_crit_cat == num_rows, t_crit_prio_3 == num_rows, 
    t_crit_prio_2 == num_prio_2, t_crit_prio_1 == num_prio_1,
    t_crit_prio_12 == num_prio_12
]

print(f'Critical cases map one to one with category?          {tests[0]}\n'
      f'Critical cases map one to one with priority=3?        {tests[1]}\n'
      f'Are all elements from priority=2 non-critical?        {tests[2]}\n'
      f'Are all elements from priority=1 non-critical?        {tests[3]}\n'
      f'Non-critical cases map one to one with priority=1,2?  {tests[2]}\n')
# --------------------------------------------------------------------------- #

**_Student comments:_**

You can write comments here.

#### **Critical state / priority analysis**

Since we have verified that the records can be decompose in three 
non-overlapping groups, we are free to inspect the arrival interval and
service times for each group and also consider the combined priority 1 and 2,
_i.e._ the non-critical patients.

Given the small size of the DataFrame, we compute and store the intervals for
each group in the dataframe, just in case this becomes useful later on.

In [ ]:
# Critical identifies a special group of patients, to which we can also
# attribute an arrival interval

# To compute differences between the elements of an array we use np.diff
# Naturally, the array of differences has one fewer element, which we 
# compensate for by appending a zero to the difference array.
# in numpy, we use np.concatenate to join to arrays
inter_1 = np.concatenate(
    ([0], np.diff(
        df.loc[idx_prio_1, 'AgentArrival'])
    )
)
inter_2 = np.concatenate(
    ([0], np.diff(
        df.loc[idx_prio_2, 'AgentArrival'])
    )
)
inter_3 = np.concatenate(
    ([0], np.diff(
        df.loc[idx_prio_3, 'AgentArrival'])
    )
)
inter_12 = np.concatenate(
    ([0], np.diff(
        df.loc[idx_prio_12, 'AgentArrival'])
    )
)

# we then add this data to the DataFrame
df.loc[idx_prio_1, 'InterPrio1'] = inter_1
df.loc[idx_prio_2, 'InterPrio2'] = inter_2
df.loc[idx_prio_3, 'InterPrio3'] = inter_3
df.loc[idx_prio_12, 'InterPrio12'] = inter_12


# note: this data is complete and contains many NaN values
df.head()

Evaluate the mean arrival and service times according to different groups

In [ ]:
# --- CODE START --- #

# Arrival time, inter arrival time (from all groups) and service time
# Group 1, 2 and 3
df.groupby('Prio')['AgentArrival', 'InterArrival', 'Service'].mean()

In [ ]:
# Arrival time, inter arrival time (from all groups) and service time
# Group 1+2
df.loc[df['Prio'] < 3.0, ['AgentArrival', 'InterArrival', 'Service']].mean()

In [ ]:
# Inter arrival time (within group)
df[['InterPrio1', 'InterPrio2', 'InterPrio3', 'InterPrio12']].mean()

# --- CODE END ----- #

**_Student comments:_**

You can write comments here.

## B. Theoretical model

The hospital considers the data to be representative and would like to 
formulate a schedule for when counters should be active.

### Exercise 1.: **Theoretical model for queuing system**

1. Develop a theoretical model for queuing system
   1. Derive theoretical properties for a Poisson queuing model of type M/M/c
      with only one arrival parameter and one service time parameter.  
   2. Estimate the parameters from the data 
   3. Calculate average utilisation, waiting time and length of queue given 6
      servers. Is such a system stable? (See hints) 

#### Exercise 1.A.: **Poisson queuing models**

1. Develop a theoretical model for queuing system
   1. Derive theoretical properties for a Poisson queuing model of type M/M/c
      with only one arrival parameter and one service time parameter.

This exercise can be divided into for tasks. Tasks 1 through 3 concern the
derivation of the mathematical model for two Markovian queuing systems and 
final task is dedicated to coding said models into Python. The task are:
1. the $M/M/1/k$ queue
2. the $M/M/c/k$ queue
3. the expectation values for the system size $\bar s$ and the queue size 
   $s_q$ for the latter queue.
4. implement the models in Python

##### 4.: **Python code for the $M/M/c/k$ queue**

Here we will define a series of Python functions to represent the analytic
solutions for the Markovian queues
- `mmck_p0`: evaluates the probability of being in state $s=0$
- `mmck_prob`: evaluates the probability of being in any state $s$
- `mmck_mean_num`: evaluates the mean system size for finite systems
   numerically
- `mmck_mean_exact`: evaluates the mean system size for any system using the
   analytic results
- `mmck_mean`: runs either of the two previous functions
- `mmck_queue_num`: evaluates the mean size of the queue for finite systems
   numerically
- `mmck_queue_exact`: evaluates the mean size of the queue for any system 
   using the analytic results
- `mmck_queue`: runs either of the two previous functions
- `wait_little`: evaluates the waiting time estimates using Little's rules

We provide incomplete definitions for all required functions, yet students
are free to write their own code from scractch if you so wish.

By default our functions will always contain the full list of arguments,
and a so-called 
[docstring](https://google.github.io/styleguide/pyguide.html#381-docstrings).

The latter includes relevant information regarding what the function does, 
which arguments/parameters it takes and what it returns (Python accepts 
functions without return).

We advise students to read the docstrings and all the comments written within
the definition of the functions.

Finally, the incomplete portions of code are highlighted as follows:
1. code that is required immediately
   ```python
   def sum(a, b):
       """Humble two number sum."""
       # ---- CODE START ---- #
       result = 
       # ---- CODE END ------ #
       return result
   ```
   where you just need to comple `result = a + b`

2.  code that is only required at latter stage
    ```python
    def sum(a, b, c=None):
        """A step beyond the humble sum."""
        # the sum of a and b
        # ---- CODE START ---- #
        result = 
        # ---- CODE END ------ #
        
        # ex.N only
        # if a third parameter is not None, add it
        # uncomment the code below and complete
        # ---- CODE START ---- #
    #    if c is not  :
    #        result 
        # ---- CODE END ------ #   
        
        return result
    ```
    
    In this case, your function will be valid as soon as you complete the 
    first part is solved immediately, replace the first definition 
    of `result =` by `result = a + b`.
    
    Once you get to `ex.N`, you have uncomment and solve the missing part of
    the code, a possible solution would read
    ```python
    # uncomment the code below and complete
    # ---- CODE START ---- #
    if c is not None:
        result += c
    # ---- CODE END ------ #   
    ```

In [ ]:
# --------------------------------------------------------------------------- #
def mmck_p0(r1, r2, c, k=None):
    """Evaluates the probability of the zeroth state being ocuppied.

    This is a numpy vectorized function over r1, r2 and c parameters.

    ```
    p0 = mmck_p0(1, 2, np.array([5, 10, 20, 50]), k=None)
    p0_exact = np.exp(-1/2)
    p0 == p0_exact
    array([False, False,  True,  True])

    p0 = mmck_p0(1, np.array([1, 2, 5, 10]), 20, k=None)
    p0_exact = np.exp(-1/np.array([1, 2, 5, 10]))
    p0 == p0_exact
    array([ True,  True,  True,  True])
    ```

    Parameters
    ----------
    r1, r2 : float, np.array of float
        arrival (lambda) and service (mu) rates
    c: int, np.array of int
        number of servers
    k : int, default=None
        queue size limit, defaults to None for the infinite queue solution

    Returns
    -------
    prob : np.ndarray of float
        probability of the zeroth state being occupied
    """
    r = r1/r2
    rbar = r1/(c*r2)

    sum1 = (math.exp(r) * special.gammaincc(c, r))

    if k == None:
      sum2 = 1 / (1 - rbar)
    else:
      sum2 = (1-rbar**(-c+k+1)) / (1-rbar)

    prob = (sum1 + ((r**c) / math.factorial(c)) * sum2)**-1

    return prob

# simple validation tests
(mmck_p0(1, 2, 20) == np.exp(-1/2),
 mmck_p0(1, 2, 20))

**_Student comments_**\
*The np.exp function rounded to a different decimal than our function, so the function is in fact correct*

In [ ]:
def mmck_prob(state, r1, r2, c, k=None):
    """Evaluates the probability of state being ocuppied.

    This function is only vectorizable over the `state` parameter.

    Parameters
    ----------
    state : int, np.ndarray of int
        state number(s)
    r1, r2 : float
        arrival (lambda) and service (mu) rates
    c: int
        number of servers
    k : int, default=None
        queue size limit, defaults to None for the infinite queue solution

    Returns
    -------
    prob : np.ndarray of float
        probability of state(s) being occupied
    """
    # it is important to use double precision numerics, i.e. 64bit ints and
    # floats

    # ---- CODE START ---- #
    r = r1/r2
    rbar = r1/(r2*c)

    if state < c:
      proba = (r**state / np.float64(special.factorial(state, exact = True))) * mmck_p0(r1, r2, c, k)
    else:
      i = np.float64(state - c)
      cfac = np.float64(special.factorial(c, exact = True))
      proba = r**(c+i) * mmck_p0(r1, r2, c, k) / (c**i * cfac)

    return proba    
    # ---- CODE END ------ #

# simple validation tests
(mmck_p0(1, 2, 20) == mmck_prob(0, 1, 2, 20),
 mmck_p0(1, 2, 20, 50) == mmck_p0(1, 2, 20),
 mmck_p0(1, 2, 20) == np.exp(-1/2),
 mmck_p0(1, 2, 20))
# --------------------------------------------------------------------------- #

In [ ]:
import scipy, numpy, math 

In [ ]:
def mmck_mean_num(r1, r2, c, k, states=None):
    """Evaluates the discrete numerical mean.
    """
    # if the state numbers array is missing, create it based on `k`
    if states is None:
        states = np.arange(k+1)

    # expectation value is just he sum of the product between
    # the state numbers and the respective probabilities
    # ---- CODE START ---- #
    probs = []
 
    for i in range (0, k+1):
      probs.append(mmck_prob(i, r1, r2, c, k))

    return np.sum(states * probs)
    
    # ---- CODE END ------ #

def mmck_mean_exact(r1, r2, c, k):
    """Evaluates the exact mean for the selected parameters"""
    r = r1 / r2
    rbar = r1 / (r2 * c)

    # evaluate the factorial of c, use scipy and make sure it is 
    # 64 bit number
    # also, compute the probabilities
    # ---- CODE START ---- #  
    
    # number_of_bytes=special.factorial(c)
    # print(number_of_bytes.nbytes) 
    
    # ---- CODE END ------ #

    # NOTE: be aware that scipy has definitions for the regularized 
    #       gamma incomplete functions, be careful
    # ---- CODE START ---- #
    s1 = r * np.exp(r) * special.gammaincc(c, r) - rbar / c / special.gamma(c)
    # expressions for s2 and s3 if k = None, ie the infinite system
    if k == None:
      s2 = rbar / (-1+rbar)**2
      s3 = 1 / (1 - rbar)
    else:
      s2 = ((c-k-1)*rbar**(-c+k+1) + (k-c)*rbar**(-c+k+2) + rbar) / (rbar - 1)**2
      s3 = (1 - rbar**(-c+k+1)) / (1 - rbar)

    prob_0 = mmck_p0(r1, r2, c, k)
    # ---- CODE END ------ #

    return prob_0 * (s1 + (r**c/math.factorial(c))*s2 + (r**c/math.factorial(c))*c*s3)

def mmck_mean(r1, r2, c, k, numeric=False, states=None):
    """Evaluates the mean expected size of queuing system using either
    discrete numerical or exact solutions.
    ```
    r1, r2, c, k = 2, 6, 1, 50
    assert mmck_mean(r1, r2, c, k, numeric=True) == mmck_mean_exact(r1, r2, c, k)
    ```
    """
    if numeric:
        return mmck_mean_num(r1, r2, c, k, states=states)
    else:
        return mmck_mean_exact(r1, r2, c, k)

# simple validation tests
r1, r2, c, k = 1, 1/0.787, 20, 100
s_num = mmck_mean(r1, r2, c, k, numeric=True)
s_exact = mmck_mean(r1, r2, c, k)
np.abs(s_num - s_exact) < 2e-16, s_exact, s_num

# --------------------------------------------------------------------------- #

In [ ]:
def mmck_queue_num(r1, r2, c, k, states=None):
    """Discrete numerical evaluation of queue size.
    """
    if states is None:
        states = np.arange(k+1)
        queuing = states - c
        queuing[:c] = 0

    # compute the probabilities for all states and then
    # evaluate the expectation for `queuing`
    # ---- CODE START ---- #
    
    probs_q=[]

    for state in states:
      probs_q.append(mmck_prob(state, r1, r2, c, k))
           
    return np.sum(queuing * probs_q) 
    # ---- CODE END ------ #

def mmck_queue_exact(r1, r2, c, k):
    """Exact evaluation of queue size."""
    r = r1 / r2
    rbar = r1 / (r2 * c)

    c_fac = np.float64(special.factorial(c, exact=True))

    # ---- CODE START ---- #

    if k == None:
      s = rbar / (1 - rbar)**2.0
    else:
      s = rbar * (1.0 - (k+1) * rbar**k + k*rbar**(k+1)) / (1 - rbar)**2.0

    prob_0 = mmck_p0(r1, r2, c, k)
    # ---- CODE END ------ #

    return s * r**c * prob_0 / c_fac

def mmck_queue(r1, r2, c, k, numeric=False, states=None):
    """Evaluates the mean size of the queue.
    """
    if numeric:
        return mmck_queue_num(r1, r2, c, k, states=states)
    else:
        return mmck_queue_exact(r1, r2, c, k)

# additional validation tests
r1, r2, c, k = 1, 2, 3, 20
sq_num = mmck_queue(r1, r2, c, k, numeric=True)
sq_exact = mmck_queue(r1, r2, c, k)
np.abs(sq_num - sq_exact) < 2e-12, sq_exact, sq_num
# --------------------------------------------------------------------------- #

In [ ]:
def wait_little(r1, r2, c, k, 
                return_error_rel=False, return_error_pc=False, 
                numeric=False):
    """Evaluates waiting times using Little's law.
lamda=r1


    $s = \lambda *W\:;\: s_q = \lambda W_q$
    $W = s / \lambda$
    $W_q = s_q / \lambda$
    $W_2 = W_q + 1/\mu$

    Estimates provided by $W$ and $W_2$ will be different for finite (and
    small) values of c and k!

W = s/Lambda

w_q= s_q/lamda
W_2=w_q+1/r2

    Returns
    -------
    w : float
        mean waiting time for the entire system
    wq : float
        mean waiting time in queue until service begins
    w2 : float
        second estimate of mean waiting time for the entire system
    rel, pc : float, optional
        relative and percentual error of w2 with respect to w1
    """

    # ---- CODE START ---- #
    
    s = mmck_mean(r1, r2, c, k, numeric)  
    s_q = mmck_queue(r1, r2, c, k, numeric)
    
    w0 = s / r1
    wq = s_q / r1
    w2 = wq + 1 / r2
    
    rel = abs(w0-w2)
    pc = abs(w0-w2)/w0

    out = w0, wq, w2

    # ---- CODE END ------ #

    return tuple(out)

# additional validation tests
r1, r2, c, k = 1, 2, 20, 50
w0, wq, w2 = wait_little(r1, r2, c, None, numeric=False)
(r1/r2 == w0,
 w0 == w2,
 (w0, wq, w2) == wait_little(r1, r2, c, 100, numeric=False))
# --------------------------------------------------------------------------- #

#### Exercise 1.B.: **Statistical analysis of data**

1. Develop a theoretical model for queuing system
    2. Estimate the parameters from the data

1. To begin, we perform a regression of the arrival and service data against
   the standard distribution for a Poisson process, namely the exponential
   distribution.
2. Then, we compare the fitted models to the data, combining a visual analysis
   of the data
3. Perfom a quantitative analsis of the fitted models using typical metrics to
   assess the quality of a regression.

##### **Useful references and notes**:

1. Students are advised to read the description of `scipy.stats`
   - [scipy.stat.fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fit.html)
   - [scipy.stat.expon](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html)
   - [scipy.stat.poisson](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html).

2. Always verify how the `loc` $(x_0)$, `scale` $(s)$ and additional 
   parameters are defined in `scipy.stats` with respect to the variable $x$.
   The general transformations is $x \to (x-x_0)/s $, hence the rates are 
   normaly defined as $\lambda, \mu \equiv 1/s$

3. There are multiple approaches to the estimation of the parameters, here we will 
   proceed as follows
   1. use `scipy.stats.fit` to fit the desired distribution via maximum 
      likelihood estimates of the parameters. The algorithm actually minimizes a
      penalized negative loglikelihood function see 
      [scipy source code](https://github.com/scipy/scipy/blob/main/scipy/stats/_fit.py#L553-L555)
   2. generate histograms for all subsets of data and plot these against the pdf
      of the respective distribution
   3. assess goodness of fit estimates to each subset

How does the model compare to data? 

Consider the mean, variance, 
skewness and excess kurtosis of the entire dataset and compare with respective
predictions from the model.

**_Student comments:_**



In [ ]:
df.head()

In [ ]:
df["InterPrio3"].unique()

In [ ]:
df1=df.copy()

In [ ]:
# not sure if we can do that because 
df1=df1.replace(np.nan,0,regex=True)
df1.head()

In [ ]:
# Creating features and labels
x1=df1.drop(columns='InterArrival')
y1=df1['InterArrival']

x2=df1.drop(columns='Service')
y2=df1['Service']

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# Scaled datasets for Arrivals and Service Predictions  
scaler = StandardScaler()

# InterArrival Times
scaler.fit(x1)
scaled_df1=scaler.transform(x1)
#print(scaled_df1)

#Service Times
scaler.fit(x2)
scaled_df2=scaler.transform(x2)
#print(scaled_df2)

In [ ]:
pca1 = PCA(n_components=2)
pca1.fit_transform(scaled_df1)
PCA1=pca1.fit_transform(scaled_df1)
print(pca1.explained_variance_ratio_)


In [ ]:
pca2 = PCA(n_components=3)
pca2.fit_transform(scaled_df2)
PCA2=pca2.fit_transform(scaled_df2)
print(pca2.explained_variance_ratio_)

In [ ]:
# Creating Scree Plot to decide how many principal components describe the most variance in our dataset
pca1=PCA().fit(scaled_df1)
plt.plot(np.cumsum(pca1.explained_variance_ratio_))
plt.xlabel("# of components")
plt.ylabel("Cumulative Explained Variance")

We can see that the first 2 principal components explain the 80% of the variance in our dataset

In [ ]:
# Creating Scree Plot to decide how many principal components describe the most variance in our dataset
pca2=PCA().fit(scaled_df2)
plt.plot(np.cumsum(pca2.explained_variance_ratio_))
plt.xlabel("# of components")
plt.ylabel("Cumulative Explained Variance")

We can see that the first 3 principal components explain more than 80% of the variance in our dataset

In [ ]:
x1=pd.DataFrame(PCA1)
x2=pd.DataFrame(PCA2)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x1,y1, test_size=0.3,random_state=12)

In [ ]:
from sklearn.linear_model import LinearRegression
OLS = LinearRegression()
OLS.fit(X_train,y_train)

In [ ]:
# Parameters
print('Intercept:'+str(OLS.intercept_))
print('Coefficients:'+str(OLS.coef_))
print('R-squared:'+str(OLS.score(X_train,y_train)))

In [ ]:
# Predictions & Errors
y_pred=OLS.predict(X_test)
perf=pd.DataFrame({"Predictions":y_pred,"Observations":y_test})
perf["Error"]=perf["Predictions"]-perf["Observations"]
perf.head()

In [ ]:
# Indexing
perf.reset_index(drop=True,inplace=True)
perf.reset_index(inplace=True)

In [ ]:
perf.head()

In [ ]:
# Plotting errors
fig=plt.figure(figsize=(25,8))
plt.bar('index','Error',data=perf,color="black",width=0.3)
plt.xlabel("Observations")
plt.ylabel('Residuals')
plt.show()

In [ ]:
# OLS with sm
import statsmodels.api as sm
Xtrain=sm.add_constant(X_train)
N_OlS=sm.OLS(y_train,X_train).fit()
N_OlS.summary()

In [ ]:
# Predicting of the service times
X_train,X_test,y_train,y_test=train_test_split(x2,y2, test_size=0.3,random_state=12)

OLS.fit(X_train,y_train)

# Parameters
print('Intercept:'+str(OLS.intercept_))
print('Coefficients:'+str(OLS.coef_))
print('R-squared:'+str(OLS.score(X_train,y_train)))

y_pred1=OLS.predict(X_test)
perf1=pd.DataFrame({"Predictions":y_pred1,"Observations":y_test})
perf1["Error"]=perf1["Predictions"]-perf1["Observations"]
perf1.head()

In [ ]:
# OLS with sm
Xtrain=sm.add_constant(X_train)
N_OlS=sm.OLS(y_train,X_train).fit()
N_OlS.summary()

##### **b. Qualitative analysis of the regression**

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import expon
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_context('notebook')

In [ ]:
# Plotting Observations
sns.distplot(df['InterArrival'], kde=True);

In [ ]:
# Plotting Observations
sns.distplot(df['InterArrival'], fit=stats.expon, kde=False);

In [ ]:
# Plotting Observations
sns.distplot(df['InterArrival'], fit=stats.expon, kde=True);

In [ ]:
# Calculate the corresponding mean 4(loc) and standard distribution(scale) for our dataset
loc, scale = stats.expon.fit(df['InterArrival'])
print('loc = {0}\tscale = {1}'.format(loc, scale))
scale_arrive = scale

In [ ]:
# Plotting Observations
sns.distplot(df['Service'],  kde=True);

In [ ]:
# Plotting Observations 
sns.distplot(df['Service'], fit=stats.expon, kde=False);

In [ ]:
# Plotting Observations
sns.distplot(df['Service'], fit=stats.expon, kde=True);

In [ ]:
#Plotting predictions of Service Times
sns.distplot(perf1["Predictions"], fit=stats.expon, kde=True);

In [ ]:
# Calculate the corresponding mean 4(loc) and standard distribution(scale) for our dataset
loc, scale = expon.fit(df['Service'])
print('loc = {0}\tscale = {1}'.format(loc, scale))
scale_service = scale

In [ ]:
# Checking Skewness in every column of our dataset
from scipy.stats import skew
from scipy.stats import kurtosis

In [ ]:
for col in df:
  print(col)
  print(skew(df[col]))

  plt.figure()
  sns.distplot(df[col])
  plt.show()

In [ ]:
for col in df:
  print(col)
  print(kurtosis(df[col]))

  plt.figure()
  sns.distplot(df[col])
  plt.show()

**_Student comments:_**



##### **c. Quantitative analysis of the regression**

Here we evaluate the Kolmogorov–Smirnov (KS) test or the $R^2$ score to
assess the goodness of the fit.

The former is a accurate metric and does not
require the creation of an arbitrary histogram, yet latter is typically more
intuitive.

**_Student comments:_**

The KS test rejects the null hypothesis in one case, the service times in
period 2 and nearly rejects it for both the daily distributions of arrival 
intervals and service times. 
This is a indicator that making approximations to the queueing model based on
Poisson process are prone to failures.

With respect to $R^2$, the results indicate acceptable agreement between most
of the observations and the respective model predictions. This is rather
surprising given the huge gap between the model the large time scale 
predictions, but it is understandable, given the probability of such events
decays exponentially and thus there impact on $R^2$ become residual.

#### Exercise 1.C.: **Average utilisation, waiting time and length of queue given** $M/M/6/k$

1. Develop a theoretical model for queuing system
   3. Calculate average utilisation, waiting time and length of queue given 6
      servers. Is such a system stable? (See hints) 

We start by defining functions that allow for the evaluation of
$P_s(r_1, r_2, c, k)$ for the general queue $M/M/c/k$.
As show in the theoretical analysis, we must first evaluate
$P_0(r_1, r_2, c, k)$

Here make use of the previously define functions for the general Markovian 
queue and Little's waiting time rules
- `mmck_p0`
- `mmck_prob`
- `mmck_mean`
- `mmck_queue`
- `wait_little`

to evaluate the average utilisation, waiting time and length of queue given 
$M/M/6/k$.

In [ ]:
# finite number of servers and infinite queue
r1 = 1/scale_arrive
r2 = 1/scale_service
c, k = 6, 1000

util = r1/(c*r2)
wait0, waitq, wait2 = wait_little(r1, r2, c, k)
queue = mmck_queue(r1, r2, c, k)

t_wait = waitq
t_total = wait0
t_queue = queue

print(util, wait0, waitq, queue)

In [ ]:
# is the MMck model stable for all cases?

# From the stability criterion, the mininum number of servers
(1/scale_arrive) / (1/scale_service)

**_Student comments:_**



## C. Queuing simulation in Python with Simpy

### Exercise 2.: Numerical queues

2. Numerical queuing models
   1. Implement the corresponding queuing model as part of an event-based
      simulator.
   2. Use it to replicate the theoretical results based on simulation.
   3. Do several runs and notice simulation variance. Consider the mean waiting
      time W. In the limit of many simulations how should the mean waiting time
      be distributed? Does the mean of this distribution converge to the
      analytical result? 

#### Exercise 2.A.: **Queues with [Simpy](https://simpy.readthedocs.io/en/latest/topical_guides/index.html)**

2. Numerical queuing models
   1. Implement the corresponding queuing model as part of an event-based
      simulator.

**Requirements for the simulator:**
- `resource_monitor`: monitors the simulator resources, _i.e._ the queue
- `patient`: defines how the patient is processed 
- `spawn_patients`: instanciates the each patient on the simulation 
   environment
- `ggck_sim`: the simulator function, sets the simulation environment for 
   generic distributions (G), with multiple servers (c) and possibly finite
   system size (k)
- `mmck_sim`: a specialized simulator for Markovian queues
- additional functions for auxiliar purposes

You are free to use data structures that are expandable, say the native `list` 
type or mutable, and also expandable, `numpy.ndarray` type. Below follows an 
example of such a function. Here, we set the function that will access the
queue resource, extract relevant information, stores it in the cache list, 
while 'returning' nothing, _i.e._ has no return statement.

**Notes:** 

1. Simpy _IS NOT_ kind to functions with return statement

   A lot of pain and anguish is to be expected from writing functions and 
   methods based the traditional setup, 
   namely `foo(*args, **kwargs) -> return`. Do not worry, python functions and
   methods do not require a return statement.

   Simpy works best if we use `cache` variables for all functions and methods
   that define simpy processes or internal definitions to processes.
   
2. Be careful with the argument names of the functions.

   The functions required for the simulator include all parameters required for
   the entire assignment. Arguments with default values are optional and 
   should cause no harm.

In [ ]:
def resource_monitor(queue, cache):
    """Single queue monitor, increments the cache list.

    Reads queue capacity, served and measures queue length, and appends
    to cache list.

    Parameters
    ----------
    queue : simpy.Resource, simpy.PriorityResource
        simpy's queuing (priority) resource
    cache : list
        list to collect queue utilization
    """
    cap = queue.capacity
    served = queue.count
    queue_len = len(queue.queue)

    cache.append([cap, served, queue_len])

# this is an auxiliar function which also makes no use of return statements
def print_if(test, msg):
    """One line conditional printing."""
    if test:
        print(msg)
# --------------------------------------------------------------------------- #

In [ ]:
def patient(env, queue, number, t_in, service, data,
            prio=0, pool=None, pool_schedule=None,
            queue_limit=None, refused=None,
            verbose=False, verbose_deep=False):
    """Define the patient process with variable priority and finite 
    pool of doctors.

    Key features of this function:
        1. check if there is space in queue for the patient and decide to
           process or reject said person

           a. process patient requires 
              1. request resource and wait until available 
                 (ex.5 onwards the request requires priority)
              2. (ex.5 only) request a doctor from the pool and wait if
                 necessary
              3. 'service' the patient, i.e. yield until the service time
                 finishes
              4. (ex.5 only) return doctors to the pool
              *. in all previous steps
                 a. record time
                 b. log relevant data

           b. refuse patient
              1. record time
              2. log basic information

    In this function, priority only affects the patient queuing order, once
    in service it longer intervenes. For intervining queues, consider
    Simpy's more general class `simpy.PreemptiveResource`.

    Queue limit throws away this patient, regardless of its priority!

    This version only allocates one doctor per patient!

    Parameters:
    -----------
    env : simpy.core.Environment
        simpy's simulation environment
    queue : simpy.Resource, simpy.PriorityResource
        simpy's queuing (priority) resource
    pool : simpy.Container
        simpy's container for dynamic quantities, this represents the pool of 
        avaliable doctors
    pool_schedule : np.ndarray of int
        number of doctors per hour
    number : int
        patient ID, which is also the arrival number
    t_in : int, float
        arrival timestamp, this is not the arrival interval
    service : int, float
        service time
    data : list of floats
        patient ID, wait for service, total time in system (wait + service)
        and priority
    prio : int, default=None
        priority value, lower values have higher priority
    queue_limit : int, default=None
        set a queue limit, if None the queue can as large as the available memory
    refused : list, default=None
        patient ID, refuse time and prio, only required if queue_limit is finite.
    verbose, verbose_deep : bool, default=False
        flags to control verbosity
    """
    # ---- CODE START ---- #
    data_patient = []
    if verbose:
      print(f'Patient {number} (with priority {prio}) arriving in ER room at {env.now}')
    with queue.request() as req:
      start = env.now
      if queue_limit != None:
        if queue.count >= queue_limit:
          refused.append([number, env.now, prio])
          if verbose:
            print(f'There is no room for patient {number} (with priority {prio}) in the ER room. They leave at {env.now}')
      else:
        # Request spot in the queue
        yield req
        data_patient.append(number)
        # Waiting time
        wait = env.now - start
        if verbose:
          print(f'Patient {number} (with priority {prio}) has been waiting {wait} minutes (at {env.now}).')
        data_patient.append(wait)
        # Service time
        yield env.timeout(service)
        if verbose:
          print(f'Patient {number} (with priority {prio}) has been getting service for {service} minutes (at {env.now}).')          
        total = wait + service
        data_patient.append(total)
        # Priority
        data_patient.append(prio)
        #Leave
        if verbose:
          print(f'Patient {number} (with priority {prio}) leaves after a total of {wait + service} minutes at {env.now}.')

        data.append(data_patient)

    # ---- CODE END ------ #

In [ ]:
def spawn_patients(env, queue, intervals, services, data,
                   priorities=None, pool=None, pool_schedule=None,
                   queue_limit=None, refused=None, 
                   verbose=False, verbose_deep=False):
    """Initializes patienties with priority and doctor pools in the 
    simulation environment.

    Key features of this function:
        1. patients are defined by equal sized arrays for intervals, service
           times and priorites (ex.5 onwards)
        2. for each patient
           a. wait for its arrival
           b. process the
            respective patient with the correct data and times

    See `patient` and `spawn_patients` for details in missing
    parameters.

    Parameters:
    ----------
    intervals, services : np.ndarray of int, float
        arrival interval and service time
    priority : np.ndarray of int, default=None
        arr
    """
    # ---- CODE START ---- #

    t_in = 0
    
    for i, interval_i in enumerate(intervals):
    

        yield env.timeout(intervals[i])

        number = int(i) + 1
        t_in += intervals[i]
        service = services[i]
        
        if priorities != None:
            prio = priorities[i]
        else:
            prio = None     

        env.process(patient(env, queue, number, t_in, service, data,
              prio=prio, 
              pool=pool, pool_schedule=pool_schedule,
              queue_limit=queue_limit,
              refused = refused,
              verbose=verbose))
              
    # ---- CODE END ------ #

In [ ]:
def ggck_sim(intervals, services, c, k=None,
               priorities=None, pool_schedule=None,
               monitor_interval=1, initial_time=0,
               verbose=False, verbose_deep=False):

    """Simple simulator for G/G/c/k queuing systems.

    Key features of this function:
        1. load simpy environments and resources
        2. process `spawn_patient` function
        3. evolve the simulation in small steps and monitor resources

    Parameters
    ----------
    intervals, services : 1d np.ndarray, list of float, int
        lists or arrays with the arrival intervals and service times in the 
        same time scale. Arrays must have equal length.
    c : int
        number of servers
    k : int, default=None
        total queue size, if None the system size is infinte in principle, 
        but eventually limited by system memory
    priorities : 1d np.ndarray, list of int, default=None
        patient priority
    pool_schedule : np.ndarray, default=None
        maximum number of doctors in the pool at any hour
    monitor_interval : int, float, default=1
        time interval for queue monitoring tool, must be positive
    initial_time : int, float, default=0
        initial simulation time
    verbose, verbose_deep : bool, default=False
        flags to control verbosity

    Returns
    -------
    data : 3d np.ndarray of float
        index of arrival, wait for service, total time in system 
        (wait + services)
    data_res : 2d np.ndarray of int
        queue capacity, service, and queue length
    refused : 2d np.ndarray of float
        index of refused arrival and respective refusal time
    """
    # ---- CODE START ---- #    
    env = simpy.Environment()
    queue = simpy.Resource(env, c)

    data=[]
    data_res=[]
    refused=[]

    env.process(spawn_patients(env, queue, intervals, services, data,
                   priorities=None, pool=None, pool_schedule=None,
                   queue_limit=k, refused=refused, 
                   verbose=False, verbose_deep=False))

    times = np.arange(1, np.sum(intervals)+1)

    for ti in times:   
      env.run(until=ti)
      resource_monitor(queue, data_res)    
    env.run()
    resource_monitor(queue, data_res)    

    np.set_printoptions(suppress = True)
                        
    data = np.array(data)
    data_res = np.array(data_res)
    refused = np.array(refused)

    return data, data_res, refused
    
    # ---- CODE END ------ #
    

In [ ]:
def mmck_sim(number, 
             p_arrival={'scale': 0.5},
             p_service={'scale': 1.0},
             c=1, k=None,
             sim_runs=1,
             dist_arrival=None, dist_service=None, rng=None,
             monitor_interval=5, initial_time=0, return_times=False,
             verbose=False, verbose_deep=False):
  
    """Run multiple simulations with exponentially distributed arrivals 
    intervals and services times.

    Makes use of the ggck queue simulator `ggck_sim` and observes its
    Parameters and Return values, so we only highlight differenteces

    Parameters
    ----------
    number : int
        number of arrivals to be considered
    p1, p2 : dict
        dictionary with required parameters for the desired distribution
    sim_runs : int, default=1
        number of simulation to be ran
    dist_arrival, dist_service : function, default=None
        numpy random number generator builtin functions for distributions,
        e.g. `rng.exponential` or `rng.gamma`. This must be consistent with
        dictionary `p1` and `p2`
    rng : np.random._generator.Generator, default=None
        numpy's random number generator to be used to create arrival
        intervals and service times

    Returns
    -------
    data : 2d/3d np.ndarray of float
        index of arrival and respective waiting times per simulation run
    data_res : list of 2d np.ndarray of int
        queue capacity, service, and queue length per simulation run
    refuses : list of 2d np.ndarray of float
        index of refused arrival and respective refusal time per simulation 
        run
    """
    # ---- CODE START ---- #
    data = []
    data_res=[]
    refused = []
    meanwaitingtimes = []
    
    for i in range(sim_runs):
      
      if rng is None:
        rng = np.random.default_rng()

      env = simpy.Environment()
      queue = simpy.Resource(env, c)

      if dist_arrival is None:
        dist_arrival = rng.exponential
      if dist_service is None:
        dist_service = rng.exponential

      intervals = dist_arrival(**p_arrival, size=number)
      services = dist_arrival(**p_service, size=number)
           
      data, data_res, refused = ggck_sim(intervals, services, c, k=k,
               priorities=None, pool_schedule=None,
               monitor_interval=1, initial_time=0,
               verbose=False, verbose_deep=False)
      
      meanwaitingtimes.append(data[:, 1].mean())
    
      np.set_printoptions(suppress = True)

    data = np.array(data)
    data_res = np.array(data_res)
    refused = np.array(refused)

    return data, data_res, refused, meanwaitingtimes
    # ---- CODE END ------ #

#### Exercise 2.B.: **Replicate theoretical results**

2. Numerical queuing models
   2. Use it to replicate the theoretical results based on simulation.`

In [ ]:
# simulate the queue with the observed arrival and service times
c = 6
k = None

# here, you only need to pass four arguments to the function
# ---- CODE START ---- #

waits_d, logs_d, refused_d = ggck_sim(df['InterArrival'], df['Service'], c, k)

print("Mean waiting time:", waits_d[:, 1].mean())
print("Mean total time:", waits_d[:, 2].mean())
print("Mean queue length:", logs_d[:, 2].mean())
# ---- CODE END ------ #

**_Student comments:_**

Results show that waiting time estimates provided by the analytic models based
on the MMck queue, considerably underestimate (up to 50%) those found by
running the data on a $G/G/c/k$ queue simulator.

#### Exercise 2.C.: **Explore results from $G/G/c/k$ and $M/M/c/k$ simulations**

2. Numerical queuing models
   3. Do several runs and notice simulation variance. Consider the mean waiting
      time W. In the limit of many simulations how should the mean waiting time
      be distributed? Does the mean of this distribution converge to the
      analytical result? 

In [ ]:
# let us visualize the results from the GGck simulation and the MMck simulation runs 0, 2

In [ ]:
# Running G/G/c/k for vizualizations
c = 6
k = None
waits_d, logs_d, refused_d = ggck_sim(df['InterArrival'], df['Service'], c, k)

In [ ]:
g_waitdf = pd.DataFrame()
g_waitdf['patientIDs'] = waits_d[:, 0]
g_waitdf['waitingtimes'] = waits_d[:, 1]
g_waitdf['totaltimes'] = waits_d[:, 2]

g_waitdf['waitingtimes_non0'] = g_waitdf['waitingtimes']
g_waitdf.loc[g_waitdf['waitingtimes_non0'] == 0.0, 'waitingtimes_non0'] = np.nan

g_waitdf['waitingtimes_non0'].max()

In [ ]:
g_logdf = pd.DataFrame()
g_logdf['capacity'] = logs_d[:, 0]
g_logdf['served'] = logs_d[:, 1]
g_logdf['queuelength'] = logs_d[:, 2]

g_logdf.head()

In [ ]:
plt.scatter(g_waitdf.index, g_waitdf['waitingtimes'])
plt.title('Waiting time in queue')
plt.xlabel("Patient ID")
plt.ylabel("Time (min)")
plt.show()

In [ ]:
plt.hist(g_waitdf['waitingtimes'], 30)
plt.title('Waiting time in queue')
plt.xlabel("Time (min)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.hist(g_waitdf['waitingtimes_non0'], 30)
plt.title('Waiting time in queue (with 0.0 excluded)')
plt.xlabel("Time (min)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.scatter(g_waitdf.index, g_waitdf['totaltimes'])
plt.title('Total time in system')
plt.xlabel("Patient ID")
plt.ylabel("Time (min)")
plt.show()

In [ ]:
plt.hist(g_waitdf['totaltimes'], 30)
plt.title('Total time in the system')
plt.xlabel("Time (min)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.scatter(g_logdf.index, g_logdf['served'])
plt.title('Number of patients served per minute')
plt.xlabel("Time interval (min)")
plt.ylabel("Served patients")
plt.show()

In [ ]:
plt.plot(g_logdf['queuelength'])
plt.title('Length of the queue per minute')
plt.xlabel("Time interval (min)")
plt.ylabel("Number of patients in the queue")
plt.show()

In [ ]:
# Running M/M/c/k for vizualizations
c = 6
k = None
number = 424
sim_runs = 100

waits_d, logs_d, refused_d, meanwaits = mmck_sim(number, 
             p_arrival={'scale': scale_arrive},
             p_service={'scale': scale_service},
             c=c, k=k,
             sim_runs=sim_runs,
             dist_arrival=None, dist_service=None, rng=None,
             monitor_interval=5, initial_time=0, return_times=False,
             verbose=False, verbose_deep=False)

In [ ]:
m_waitdf = pd.DataFrame()
m_waitdf['patientIDs'] = waits_d[:, 0]
m_waitdf['waitingtimes'] = waits_d[:, 1]
m_waitdf['totaltimes'] = waits_d[:, 2]

m_waitdf['waitingtimes_non0'] = m_waitdf['waitingtimes']
m_waitdf.loc[m_waitdf['waitingtimes_non0'] == 0.0, 'waitingtimes_non0'] = np.nan

m_waitdf.head()

In [ ]:
m_logdf = pd.DataFrame()
m_logdf['capacity'] = logs_d[:, 0]
m_logdf['served'] = logs_d[:, 1]
m_logdf['queuelength'] = logs_d[:, 2]

m_logdf.head()

In [ ]:
print("M/M/c/k theoretical model")
print("Mean waiting time:", round(t_wait,2))
print("Mean total time:", round(t_total,2))
print("Mean queue length:", round(t_queue,2))
print("")
print("G/G/c/k sim")
print("Mean waiting time:", round(g_waitdf['waitingtimes'].mean(),2))
print("Mean total time:", round(g_waitdf['totaltimes'].mean(),2))
print("Mean queue length:", round(g_logdf['queuelength'].mean(),2))
print("")
print("M/M/c/k sim")
print("Mean waiting time:", round(m_waitdf['waitingtimes'].mean(),2))
print("sd:", round(m_waitdf['waitingtimes'].std(),2))
print("Mean total time:", round(m_waitdf['totaltimes'].mean(),2))
print("sd:", round(m_waitdf['totaltimes'].std(),2))
print("Mean queue length:", round(m_logdf['queuelength'].mean(),2))
print("sd:", round(m_logdf['queuelength'].std(),2))

In [ ]:
sns.distplot(meanwaits, fit=stats.norm,  kde=True);

**_Student comments:_**



**_Student comments:_**



### Exercise 3.: **Service team size analysis**

3. Service teams size 
   1. Run simulations with server teams ranging from 3 to 9 elements based on
      exercise the general queue $G/G/c/k$.
   2. Is the queue stable?
   3. Plot the corresponding waiting-time distributions of the system. 
      Describe the changes. Also, plot the mean waiting time as a function of
      the number of servers. How does the marginal value of adding more 
      servers develop? From a pure efficiency argument how many servers should
      the hospital employ?

#### Exercise 3.A.: **Run simulations**

3. Service teams size 
   1. Run simulations with server teams ranging from 1 to 9 elements based on
      exercise the general queue $G/G/c/k$.

In [ ]:
waits_ds = []
logs_ds = []
refused_ds = []

# ---- CODE START ---- #
# load data from the data frame
arrivals = df['InterArrival'] 
services = df['Service']

# create a numpy array with numbers 1 to 9 inclusive!
c_s = np.array(np.arange(1,10))
for c in c_s:
    # you only need four parameters for these simulations
    waits_d, logs_d, refused_d = ggck_sim(arrivals, services, c, k=None)
    avg_wait = waits_d[:, 1].mean()
    avg_total = waits_d[:, 2].mean()
    avg_queue = logs_d[:, 2].mean()
    print(c, round(avg_total, 2), round(avg_wait,2), round(avg_queue,2))
    
# ---- CODE END ------ #

**_Student comments:_**



#### Exercise 3.B.: **Stability**

3. Service teams size 
   2. Is the queue stable?

In [ ]:
# here we simulate all queues from 1 to 12 servers during 7 days
waits_ds = []
logs_ds = []
refused_ds = []

# ---- CODE START ---- #
c_s = np.array(np.arange(1,13))
num_days = 7

# we have data for one full day, let's repeat it over multiple days
# numpy has a function that makes tiling periodic systems trivial
arrivals_day = df['InterArrival'] 
arrivals_week = np.tile(arrivals_day, num_days)
services_day = df['Service']
services_week = np.tile(services_day, num_days)

arrival_rate = df['InterArrival'].mean() 
service_rate = df['Service'].mean()       

avg_waits=[]
avg_totals=[]

wait_per_c = pd.DataFrame()
total_per_c = pd.DataFrame()
queue_per_c = pd.DataFrame()

for c in c_s:
    # again, four parameters only
    waits_d, logs_d, refused_d = ggck_sim(arrivals_week, services_week, c, k=None)

    avg_wait = waits_d[:, 1].mean()
    avg_total = waits_d[:, 2].mean()
    avg_queue = logs_d[:, 2].mean()
    
    avg_waits.append(avg_wait)
    avg_totals.append(avg_total)

    wait_per_c[str(c)] = waits_d[:, 1]
    total_per_c[str(c)] = waits_d[:, 2]
    queue_per_c[str(c)] = logs_d[:, 2]


    utilization = (1/arrival_rate)/(c*(1/service_rate))
    
    print(c, round(utilization,2), round(avg_total, 2), 
          round(avg_wait,2), round(avg_queue,2))
        
# ---- CODE END ------ #

In [ ]:
wait_servers=pd.DataFrame()
wait_servers["avg_waits"]=avg_waits
wait_servers["avg_totals"]=avg_totals
wait_servers["servers"]=c_s

In [ ]:
plt.figure(figsize=(20, 5))
for c in range(1,13):
  plt.plot(queue_per_c[str(c)],label=c)
plt.ylim((0,70))
plt.title('Length of the queue per minute')
plt.xlabel("Time interval (min)")
plt.ylabel("Number of servers")
plt.legend()
plt.show()

**_Student comments:_**

You can write comments here.

#### Exercise 3.C.: **Waiting times and marginal value**

3. Service teams size 
   3. Plot the corresponding waiting-time distributions of the system. 
      Describe the changes. Also, plot the mean waiting time as a function of
      the number of servers. How does the marginal value of adding more 
      servers develop? From a pure efficiency argument how many servers should
      the hospital employ?

In [ ]:
# Plotting waiting-time distributions of the system
wait_per_c.plot.density(figsize = (7, 7), logx=True, xlim=(0, 10000), linewidth = 4)

In [ ]:
# Plotting average waiting time as a function of servers
sns.lineplot(data=wait_servers, x="servers", y="avg_waits")

In [ ]:
# Plotting average waiting time as a function of servers
sns.lineplot(data=wait_servers, x="servers", y="avg_totals")

In [ ]:
c=6
waits_d, logs_d, refused_d = ggck_sim(arrivals_day, services_day, c, k=None)

wait_servers=pd.DataFrame()
wait_servers["waiting_time"]=waits_d[:,1]
wait_servers["service_time"]=waits_d[:,2]-waits_d[:,1]



In [ ]:
# Computing quantiles for a week
print("0.90 quantile of  waiting for service time:" ,wait_servers["service_time"].quantile(0.90))
print("0.95 quantile of  waiting for service time:" ,wait_servers["service_time"].quantile(0.95))
print("0.99 quantile of  waiting for service time:" ,wait_servers["service_time"].quantile(0.99))

print("0.90 quantile of  total time:" ,wait_servers["waiting_time"].quantile(0.90))
print("0.95 quantile of  total time:" ,wait_servers["waiting_time"].quantile(0.95))
print ("0.99 quantile of  total time:" ,wait_servers["waiting_time"].quantile(0.99))



**_Student comments:_**



## D. Queuing and priority

### Exercise 4.: **Queuing and priority**

4. Queuing and priority. The hospital has been informed about the merits of 
   the [Manchester Triage System](https://www.triagenet.net/classroom/) and
   wants to implement a similar system
   1. Implement a priority queue system, such that critical patients have the
      highest priority and are served first (prio =3), children below the age
      of 16 have second highest priority (prio = 2), while everybody else have
      lowest priority (prio = 1).  
   2. Measure the waiting time impact for these three classes of people
      compared to the First-come-first-serve principle from before?

#### Exercise 4.A: **Priority queues with Simpy**

4. Queuing and priority.
   1. Implement a priority queue system, such that critical patients have the
      highest priority and are served first (prio =3), children below the age
      of 16 have second highest priority (prio = 2), while everybody else have
      lowest priority (prio = 1).
      
This exercise is a generalization of exercise 2.A and we will modified the
relevant functions to make use of more elaborate queue, the 
`simpy.PriorityResource`, to introduce the required functionality.

Please return to the code writen in ex.2.A and address all comments with
reference to ex.6.

Functions that require modifications:
- `patient`: defines how the patient is processed 
- `spawn_patients`: instanciates the each patient on the simulation 
   environment
- `ggck_sim`: the simulator function, sets the simulation environment for 
   generic distributions (G), with multiple servers (c) and possibly finite
   system size (k)

In [ ]:
def patient(env, queue, number, t_in, service, data,
            prio=0, pool=None, pool_schedule=None,
            queue_limit=None, refused=None,
            verbose=False, verbose_deep=False):
    # ---- CODE START ---- #
    data_patient = []
    
    if verbose:
      print(f'Patient {number} (with priority {prio}) arriving in ER room at {env.now}')

    with queue.request(priority = prio) as req:
      start = env.now
      
      if queue_limit != None:
        if queue.count >= queue_limit:
          refused.append([number, env.now, prio])
          if verbose:
            print(f'There is no room for patient {number} (with priority {prio}) in the ER room. They leave at {env.now}')
        
      else:
                
        # Request spot in the queue
        yield req
        data_patient.append(number)

        # Waiting time
        wait = env.now - start
        
        if verbose:
          print(f'Patient {number} (with priority {prio}) has been waiting {wait} minutes (at {env.now}).')
        
        data_patient.append(wait)

        # Service time
        yield env.timeout(service)
        if verbose:
          print(f'Patient {number} (with priority {prio}) has been getting service for {service} minutes (at {env.now}).')          
        total = wait + service
        data_patient.append(total)

        # Priority
        data_patient.append(prio)
        
        #Leave
        if verbose:
          print(f'Patient {number} (with priority {prio}) leaves after a total of {wait + service} minutes at {env.now}.')
        
        data.append(data_patient)

    # ---- CODE END ------ #

In [ ]:
def spawn_patients(env, queue, intervals, services, data,
                   priorities=None, pool=None, pool_schedule=None,
                   queue_limit=None, refused=None, 
                   verbose=False, verbose_deep=False):

    # ---- CODE START ---- #

    t_in = 0
    
    for i, interval_i in enumerate(intervals):

        yield env.timeout(intervals[i])

        number = int(i) + 1
        t_in += intervals[i]
        service = services[i]
        
        prio=priorities[i]     

        env.process(patient(env, queue, number, t_in, service, data,
              prio=prio, 
              pool=pool, pool_schedule=pool_schedule,
              queue_limit=queue_limit,
              refused = refused,
              verbose=verbose))
              
    # ---- CODE END ------ #

In [ ]:
def ggck_sim(intervals, services, c, k=None,
               priorities=None, pool_schedule=None,
               monitor_interval=1, initial_time=0,
               verbose=False, verbose_deep=False):

    # ---- CODE START ---- #
    
    env = simpy.Environment()
    queue = simpy.PriorityResource(env, c)

    data=[]
    data_res=[]
    refused=[]

    env.process(spawn_patients(env, queue, intervals, services, data,
                   priorities=priorities, pool=None, pool_schedule=None,
                   queue_limit=k, refused=refused, 
                   verbose=False, verbose_deep=False))

    times = np.arange(1, np.sum(intervals)+1)

    for ti in times:
      env.run(until=ti)
      resource_monitor(queue, data_res)    
    env.run()
    resource_monitor(queue, data_res)    

    np.set_printoptions(suppress = True)
                        
    data = np.array(data)
    data_res = np.array(data_res)
    refused = np.array(refused)

    return data, data_res, refused
    
    # ---- CODE END ------ #

#### Exercise 4.B: **Priority vs non-priority**

6. Queuing and priority.
   2. Measure the waiting time impact for these three classes of people
      compared to the First-come-first-serve principle from before?

In [ ]:
priorities = 1 -1 * df['Prio'].values  # The smaller the number, the higher is its priority!

c, k = 6, None

x1,x2,x3 = ggck_sim(df['InterArrival'], df['Service'], c, k,
               priorities=priorities, pool_schedule=None,
               monitor_interval=1, initial_time=0,
               verbose=False, verbose_deep=False)

In [ ]:
prio_waitdf = pd.DataFrame()
prio_waitdf['patientIDs'] = x1[:, 0]
prio_waitdf['waitingtimes'] = x1[:, 1]
prio_waitdf['totaltimes'] = x1[:, 2]
prio_waitdf['priorities'] = x1[:, 3]

prio_logdf = pd.DataFrame()
prio_logdf['capacity'] = x2[:, 0]
prio_logdf['served'] = x2[:, 1]
prio_logdf['queuelength'] = x2[:, 2]

In [ ]:
print("Mean waiting time:")
print(prio_waitdf.groupby('priorities')['waitingtimes'].mean())
print("")
print("Mean total time:")
print(prio_waitdf.groupby('priorities')['totaltimes'].mean())
print("")
print("Mean queue length:")
print(prio_logdf['queuelength'].mean())

### Exercise 5.: **Priority and waiting time differentiation**

7. The hospital decides that waiting time goals will be different for each
   priority group, namely: category 2 patients must at most wait 3 minutes, 
   while category 1 patients have a 95% percentile waiting time of maximum 15
   minutes. (95% of the incoming people should be served before the 15th 
   minute). Priority queues are still in use.
   1. Implement variable size queue in Python
   2. Find a capacity schedule such that none of the constraints are violated
      for a schedule with three intervals of 8 hours each starting from 
      midnight [0-8,8-16,16-24] and present proper KPIs for the system during
      the period.  
   3. If waiting time is evaluated by 1000 DKK/H when below the 3 and 15
      minute threshold, and 5000 DKK/H when above (constraint is violated),
      what is the accumulated waiting time cost over a day for a capacity 
      schedule of [6,6,6]? What is the cost for your schedule found in 7.a? 
   4. (OPTIONAL) The cost of an active server is 1000 DKK/H. Try to find a 
      ‘good’ capacity mix over the three time-slots. You may base your
      assessment on the basis of a dedicated objective function or other
      measures of performance.  

#### Exercise 5.A: **Capacity scheduling in Python**

Just as in ex.4.A, this is a generalization of exercise 2.A and we will 
modified the relevant functions to introduce the required functionality. 

It is important to note that, Simpy does not allow the capacity of the 
resource queue to vary, but offer an additional class, `simpy.Container`, that
allows us to solve this problem.

Functions that require modifications:
- `patient`: defines how the patient is processed 
- `spawn_patients`: instanciates the each patient on the simulation 
   environment
- `ggck_sim`: the simulator function, sets the simulation environment for 
   generic distributions (G), with multiple servers (c) and possibly finite
   system size (k)


In [ ]:
def patient(env, queue, number, t_in, service, data,
            prio=0, pool=None, pool_schedule=None,
            queue_limit=None, refused=None,
            verbose=False, verbose_deep=False):
    # ---- CODE START ---- #
    data_patient = []

    if verbose:
      print(f'Patient {number} (with priority {prio}) arriving in ER room at {env.now}')

    with queue.request(priority = prio) as req:
      start = env.now
      if queue_limit != None:
        if queue.count >= queue_limit:
          refused.append([number, env.now, prio])
          if verbose:
            print(f'There is no room for patient {number} (with priority {prio}) in the ER room. They leave at {env.now}')
        
      else:
        
        yield req
        data_patient.append(number)

        # Request doctor
        yield pool.get(1)

        # Waiting time
        wait = env.now - start                    
        if verbose:
          print(f'Patient {number} (with priority {prio}) has been waiting {wait} minutes (at {env.now}).')
        data_patient.append(wait)

        # Service time
        yield env.timeout(service)
        if verbose:
          print(f'Patient {number} (with priority {prio}) has been getting service for {service} minutes (at {env.now}).')          
        total = wait + service
        data_patient.append(total)

        # Priority
        data_patient.append(prio)
        
        #Leave
        if verbose:
          print(f'Patient {number} (with priority {prio}) leaves after a total of {wait + service} minutes at {env.now}.')
        
        data.append(data_patient)

        #Return doctor to pool
        pool_level = pool.level
        schedule_index = math.ceil(env.now / 60) - 1
        schedule_index = schedule_index % 23
        pool_size = pool_schedule[schedule_index]


        if pool_level < pool_size:
          yield pool.put(1)

    # ---- CODE END ------ #

In [ ]:
def spawn_patients(env, queue, intervals, services, data,
                   priorities=None, pool=None, pool_schedule=None,
                   queue_limit=None, refused=None, 
                   verbose=False, verbose_deep=False):

    # ---- CODE START ---- #

    t_in = 0
    
    for i, interval_i in enumerate(intervals):
    

        yield env.timeout(intervals[i])

        number = int(i) + 1
        t_in += intervals[i]
        service = services[i]

        prio = priorities[i]    

        env.process(patient(env, queue, number, t_in, service, data,
              prio=prio, 
              pool=pool, pool_schedule=pool_schedule,
              queue_limit=queue_limit,
              refused = refused,
              verbose=verbose))
              
    # ---- CODE END ------ #

In [ ]:
def doctor_fill(env, queue, pool, pool_schedule, 
                   verbose=False, verbose_deep=False):
    pool_level = pool.level
    schedule_index = math.ceil(env.now / 60) - 1
    schedule_index = schedule_index % 23

    if schedule_index == 8:
      if pool_schedule[8]-pool_schedule[7] > 0:
        yield pool.put(pool_schedule[8]-pool_schedule[7])
    if schedule_index == 16:
      if pool_schedule[16]-pool_schedule[15] > 0:
        yield pool.put(pool_schedule[16]-pool_schedule[15])

In [ ]:
def ggck_sim(intervals, services, c, k=None,
               priorities=None, pool_schedule=None,
               monitor_interval=1, initial_time=0,
               verbose=False, verbose_deep=False):

    # ---- CODE START ---- #   
    env = simpy.Environment()
    queue = simpy.PriorityResource(env, c)
    pool = simpy.Container(env, c, init=pool_schedule[0]) 

    data=[]
    data_res=[]
    refused=[]

    env.process(spawn_patients(env, queue, intervals, services, data,
                   priorities=priorities, pool=pool, pool_schedule=pool_schedule,
                   queue_limit=k, refused=refused, 
                   verbose=False, verbose_deep=False))
    env.process(doctor_fill(env, queue, pool, pool_schedule, 
                   verbose=False, verbose_deep=False))

    times = np.arange(1, np.sum(intervals)+1)

    for ti in times:   
      env.run(until=ti)
      resource_monitor(queue, data_res)    
    env.run()
    resource_monitor(queue, data_res)    

    np.set_printoptions(suppress = True)
                        
    data = np.array(data)
    data_res = np.array(data_res)
    refused = np.array(refused)

    return data, data_res, refused
    
    # ---- CODE END ------ #

#### Exercise 5.B: **The impact of scheduling**

5. Priority and waiting time differentiation
   2. Find a capacity schedule such that none of the constraints are violated
      for a schedule with three intervals of 8 hours each starting from 
      midnight [0-8,8-16,16-24] and present proper KPIs for the system during
      the period.  

In [ ]:
priorities = 1 -1 * df['Prio'].values  # The smaller the number, the higher is its priority!

intervals = df['InterArrival'].values
services = df['Service'].values

pool_wait = pd.DataFrame()

# CONSTRAINTS:
# category 2 (-1) patients must at most wait 3 minutes, 
# while category 1 (0) patients have a 95% percentile 
# waiting time of maximum 15 minutes. 
# (95% of the incoming people should be served before the 15th minute)

first_shift = [11]
second_shift = [9]
third_shift = [9]

first_shift = np.tile([first_shift], 8)
second_shift = np.tile(second_shift, 8)
third_shift = np.tile(third_shift, 8)
pool_schedule = np.append(first_shift, second_shift)
pool_schedule = np.append(pool_schedule, third_shift)

c, k = 100, None

x1,x2,x3 = ggck_sim(intervals, services, c, k,
              priorities=priorities, pool_schedule=pool_schedule,
              monitor_interval=1, initial_time=0,
              verbose=False, verbose_deep=False)

pool_waitdf = pd.DataFrame()
pool_waitdf['patientIDs'] = x1[:, 0]
pool_waitdf['waitingtimes'] = x1[:, 1]
pool_waitdf['priorities'] = x1[:, 3]

print("First constraint: (about prio -1)")
print(pool_waitdf.groupby('priorities')['waitingtimes'].max())
print("Second constraint: (about prio 0)")
print(pool_waitdf.groupby('priorities')['waitingtimes'].quantile(0.95))


#### Exercise 5.C: **Cost based analysis**

5. Priority and waiting time differentiation
   2. If waiting time is evaluated by 1000 DKK/H when below the 3 and 15
      minute threshold, and 5000 DKK/H when above (constraint is violated),
      what is the accumulated waiting time cost over a day for a capacity 
      schedule of [6,6,6]? What is the cost for your schedule found in 7.a? 

In [ ]:
priorities = 1 -1 * df['Prio'].values  # The smaller the number, the higher is its priority!

intervals = df['InterArrival'].values
services = df['Service'].values

pool_wait = pd.DataFrame()

# CONSTRAINTS:
# category 2 (-1) patients must at most wait 3 minutes, 
# while category 1 (0) patients have a 95% percentile 
# waiting time of maximum 15 minutes. 
# (95% of the incoming people should be served before the 15th minute)

first_shift = [11]
second_shift = [9]
third_shift = [9]

# first_shift = [6]
# second_shift = [6]
# third_shift = [6]

first_shift = np.tile([first_shift], 8)
second_shift = np.tile(second_shift, 8)
third_shift = np.tile(third_shift, 8)
pool_schedule = np.append(first_shift, second_shift)
pool_schedule = np.append(pool_schedule, third_shift)

c, k = 100, None

x1,x2,x3 = ggck_sim(intervals, services, c, k,
              priorities=priorities, pool_schedule=pool_schedule,
              monitor_interval=1, initial_time=0,
              verbose=False, verbose_deep=False)

pool_waitdf = pd.DataFrame()
pool_waitdf['waitingtimes'] = x1[:, 1]
pool_waitdf['priorities'] = x1[:, 3]

cost = 0

for i in range(len(pool_waitdf['waitingtimes'])):
  if pool_waitdf['priorities'][i] == -1:
    if pool_waitdf['waitingtimes'][i] < 3:
      cost += 1000*pool_waitdf['waitingtimes'][i]/60
    else:
      cost += 5000*pool_waitdf['waitingtimes'][i]/60
  if pool_waitdf['priorities'][i] == 0:
    if pool_waitdf['waitingtimes'][i] < 15:
      cost += 1000*pool_waitdf['waitingtimes'][i]/60
    else:
      cost += 5000*pool_waitdf['waitingtimes'][i]/60
  if pool_waitdf['priorities'][i] == -2:
    cost += 1000*pool_waitdf['waitingtimes'][i]/60

print(cost)

#### Exercise 5.D: **Cost based optimum**

5. Priority and waiting time differentiation
   3. (OPTIONAL) The cost of an active server is 1000 DKK/H. Try to find a 
      ‘good’ capacity mix over the three time-slots. You may base your
      assessment on the basis of a dedicated objective function or other
      measures of performance.  